In [3]:
Packages <- c("dplyr", "R.utils", "nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum1.R") #integrate_kernel_sum0.5
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve.R") #median
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve90.R") #dist 90% retained

#read in the kernel fitting summary
kernels <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv")
#read in the centroids adjusted for the simulation, so the Magbangons combined 
centroids <- fread(file="~/oceanography/empirical_data/site_centroids_SimTest.csv")
setorder(centroids, site)
#read in the table with number of recruits sampled at each site for each year
AnnualRecsSamp <- fread(file="~/oceanography/script_output/SurveyData/AnnualRecruitsSampled.csv")
#read in the table of the proportion of anemones sampled at each site for each year
PropSamp <- fread(file="~/oceanography/script_output/SurveyData/ProportionHabitatSampled.csv")

#read in the ROMS simulation connectivity table
SimConn <- fread(file="~/oceanography/script_output/ROMSDataTables/SimConnectivityTable.csv")

#make vectors defining sites we didn't sample, but that are in the model, and the sandflats specifically 
UnsampledSites <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan", "Other", "CAI") 
SandFlats <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6") 

#make the constant inputs for the kernel fitting function
#distance matrix using the centroids with combined Magbangon
### List of source locations
sites_source <- centroids

### List of destination locations
sites_dest <- centroids

dist_mat_m <- distm(sites_source[,c('lon','lat')], sites_source[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- dist_mat_m*10^-3
#read in the reef areas for the kernel fitting
Area <- fread("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels_test.csv") %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6)# %>%
    #select(kmsq) #need to uncomment for functions to work
setorder(Area, site)
Reef_sizes <- as.matrix(Area$kmsq)

#make a table with the survey information for each site (how many fish sampled, prop anems sampled, total number of anems at site)
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year=end_year, site)][#join the sampling tables together
    is.na(n_offs_gen), n_offs_gen := 0][#change NA's to 0
    ,-"time_frame"]#drop the time_frame column, we can key with end_year
#setkey(SurveyData, site)

__Can skip the cells below and read in the already formatted dest_sampled table.__


In [4]:
#make the connectivity data long form
SimConn <- SimConn[destination !="Other" & source %!in% SandFlats][ #we don't care about "other" as a destination, and sand flats can't be a source
    ,!c("SourceSampled", "DestSampled")]#drop these columns, it was used for sorting in the process_ROMS script but at this point it's confusing
check1 <- sum(SimConn$DailyParticles)   
#setkey(SimConn, source)
SimConn <- SimConn[rep(seq_len(nrow(SimConn)), SimConn$DailyParticles), 1:ncol(SimConn)][ #make long form, repeat each row the number of times specified in "DailyParticles"
    , ParticleID := paste ("P", .I, sep = "", collapse = NULL)]#finally, make a new column that's the particle ID for each row

nrow(SimConn)==check1 #should be TRUE, meaning the correct number of particles are retained

#the below code is subsampling particles, but we want to do that within the loop of kernel fitting
#SimConnDTDir3 <- SimConnDTDir2[c(SimConnDTDir2[, .I[source != "Other"]], sample(SimConnDTDir2[, .I[source == "Other"]], length(SimConnDTDir2[, .I[source == "Other"]])*.1))]
#SimConnDTDir3 <- SimConnDTDir2[c(SimConnDTDir3[, .I[source != "CAI"]], sample(SimConnDTDir3[, .I[source == "CAI"]], length(SimConnDTDir3[, .I[source == "CAI"]])*.1))]


[1] TRUE

In [ ]:
SimConn[, .(.N), by = .(source)]

In [26]:
##make a table with the survey information for each site (how many fish sampled, prop anems sampled, total number of anems at site)
#SurveyData <- AnnualRecsSamp[PropSamp, on=.(year=end_year, site)][#join the sampling tables together
#    is.na(n_offs_gen), n_offs_gen := 0][#change NA's to 0
#    ,-"time_frame"]#drop the time_frame column, we can key with end_year
##setkey(SurveyData, site)
#

In [5]:
#join the simulation data with the survey data
source_join <- SurveyData[SimConn, on = .(site = source, year=YearSampled)]
setnames(source_join, c("site", "year", "n_offs_gen", "PropAnemSamp", "TotalAnems"), c("Source","YearSampled", "SourceNOffsAnnual",  "SourcePropSamp", "SourceTotalAnems"))

destination_join <- SurveyData[source_join, on = .( site= destination, year=YearSampled)]
setnames(destination_join, c("site", "year", "n_offs_gen", "PropAnemSamp", "TotalAnems"), c("Dest","YearSampled", "DestNOffsAnnual",  "DestPropSamp", "DestTotalAnems"))



In [60]:
#lets see if we can do all the kernel fitting in one loop! be ambitious! this table created here will be what we loop through for kernel fitting
SimDates <- destination_join[,.(date= unique(date)), by=.(YearSampled)][ #make a data.table of dates in that year-
    , DateSeqLin := ((0.25/2)/5)*.I, by=.(YearSampled)][ #then add a column of row number mulitplied by the linear survival model slope
    , SurvWeight := DateSeqLin/(max(DateSeqLin)*1.25), by=.(YearSampled)][ #then add a column normalizing that as the survival probability, BUT if not eliminating new recruits (months 4/5) multiply the denominator by 1.25 so that nothing is an arbitrary prob=1
    ,-"DateSeqLin"]

#check this looks right- 2014 has fewer dates than the other years because the simulation is truncated for that time, and should see the SurvWeight column get low again when the year changes
#SimDates[, .(.N), by = .(YearSampled)]
#SimDates[240:250,]


#now join with the simulation data table
date_join <- SimDates[destination_join, on=.(date, YearSampled)][#joining with YearSampled too is redundant, can otherwise drop the column from SimDates before joining #
    SimMonth %!in% c(4,5)] #filter out particles recruiting in April/May, they would be new recruits
 
nrow(date_join)==nrow(SimConn[SimMonth %!in% c(4,5)]) #should be TRUE

#finally, weight the sources by their number of anemones relative to the total number of anemones across the surveyed region
total_anems <- sum(SurveyData[year=="2014"][,"TotalAnems"]) #get the total number of anems recorded for the simulation time period
date_join <- date_join[, SourceSize := SourceTotalAnems/total_anems, by=.(Source)]



[1] TRUE

In [63]:
#save inter file
fwrite(date_join, file="~/oceanography/script_output/ROMSDataTables/LongFormConnWithProbs.csv")
#read in the inter file instead of re-running above code
#date_join <- fread(file="~/oceanography/script_output/ROMSDataTables/LongFormConnWithProbsTest.csv")

In [61]:
#outside of the loop, trim this to only be the destinations we sampled

dest_sampled <- date_join[DestPropSamp >0]
dest_sampled <- kernels[Year %in% c("2012", "2013", "2014")][, Year:=as.integer(Year)][,c("Year", "NumParentageMatches")][dest_sampled, on=.(Year=YearSampled)]#[
    #, ParticleID := paste ("P", .I, sep = "", collapse = NULL)]#finally, make a new column that's the particle ID for each row

#each year will require a different set of survey data, so make a list of each and index by site for fast look up
PropSampTable <- SurveyData[PropAnemSamp >0, c("year", "site")]#[
    #, c("Year", "Source", "Dest", "Parentage")] #only grab the columns we need
#setkey(year)

#make a site index table, use this for Sampled_reefs input in kernel fitting
site_index <- unique(centroids, by="site")[, "site"][, index := .I] #add the row number as the unique site index

#make sure all sampled sites are represented by joining the survey data to the sampled simulation
PropSampTable <- rbind(SurveyData[PropAnemSamp >0 & year %in% c(2012, 2013, 2014), c("year", "site")][, .(Source=site, Dest=site, Year=year)][ #will join to the simulated sampling table by source and dest, so make those each a column from site and preserve the year variable as a key
    , c("Year", "Source", "Dest")][, Monsoon := "NEM"], SurveyData[PropAnemSamp >0 & year %in% c(2012, 2013, 2014), c("year", "site")][, .(Source=site, Dest=site, Year=year)][ #will join to the simulated sampling table by source and dest, so make those each a column from site and preserve the year variable as a key
    , c("Year", "Source", "Dest")][, Monsoon := "SWM"])
unq_survey <- unique(PropSampTable, by=c("Source", "Dest", "Year", "Monsoon"))#, unique(PropSampTable, by=c("Source", "Dest", "Year"))[, Monsoon := "SWM"]) #add in the diff Monsoon seasons so there are complete parentage matrices later
add_dest <- rbind(SurveyData[year %in% c(2012, 2013, 2014) & PropAnemSamp >0][, c("year", "site")][, Monsoon := "NEM"], SurveyData[year %in% c(2012, 2013, 2014) & PropAnemSamp >0][, c("year", "site")][, Monsoon := "SWM"])  #what destinations were sampled, for use with unassigned table

#rename the monsoon column in the full table for consistency
setnames(dest_sampled, "SimMonsoon", "Monsoon")



In [66]:
summary(dest_sampled$SurvWeight)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4016  0.6014  0.6540  0.6547  0.7251  0.7845 

In [73]:
#make an empty data.table to hold all of the simulation results
#row_num <- 4*1000
SimKernels <- data.table(TimeScale=character(), TimeID=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric(), iteration=numeric())

#pb <- txtProgressBar(min = 0, max = 10, style = 3)

#StartTime <- Sys.time()


#for(i in 1:10){

#sample the particle data
sim_sample <- dest_sampled[, .SD[sample(.N, DestNOffsAnnual, prob=SurvWeight)], by = c("Year", "Dest")] #randomly sample rows (particles) from the table according to the survival weighting, based on the number we sampled at each site in each year of surveys
check1 <- nrow(sim_sample)

#assign parentage
sim_parentage <- sim_sample[SourcePropSamp > 0][, .SD[sample(.N, NumParentageMatches, prob=SourcePropSamp)], by = .(Year)][#now randomly assign parentage or not parentage, based on how well we sampled the source and the number of parentage matches we had in that year
                , Parentage := 1]
#for faster searching, set keys
setkey(sim_parentage, ParticleID)
setkey(sim_sample, ParticleID)

l <- list(sim_sample[ParticleID %!in% sim_parentage$ParticleID][, Parentage := 0], sim_parentage)
sim_sample <- rbindlist(l, use.names = TRUE, fill=TRUE, idcol = NULL)[, c("Year", "Source", "Dest", "Parentage", "Monsoon")] #add back in to the unassigned particles, select only the columns necessary

#check results, for testing loop only
nrow(sim_sample)==check1 #should be TRUE
sum(sim_sample$Parentage) #should be 37

#calculate the unassigned row
unassigned <- unique(sim_sample[Parentage==0][#not counting parentage!
    , num_sampled := .(.N), by= c("Dest", "Year", "Monsoon")], by=c("Dest", "Year", "Monsoon"))[, -"Source"]
#add destinations not sampled in loop iteration to unassigned 
unassigned <- unassigned[add_dest, on=.(Year=year, Dest=site, Monsoon)]
unassigned$num_sampled[is.na(unassigned$num_sampled)] <- 0
sum(unassigned$num_sampled, na.rm=T)==check1-37 #total should be the total sampled particles minus the total assigned
setorder(unassigned, Year, Dest)

##adding in the possible sampled routes needs to happen AFTER calculating unassigned because unassigned is calculated from row counts
sim_sample <- PropSampTable[sim_sample, on=.(Year, Source, Dest, Monsoon)]
#check all is well- for testing loop only
sum(sim_sample$Parentage) #should be 37

#add in the routes we could have assigned given our sampling so the parentage matrix is complete
unq_sim_sample <- unique(sim_sample, by=c("Source", "Dest", "Year", "Monsoon"))

add_routes <- unq_survey[!unq_sim_sample, on = names(unq_survey)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , `:=`(Parentage= 0, num_sampled = 0) ] #add the parentage column 

#add back into the sampled simulation data
l <- list(sim_sample, add_routes[,-"num_sampled"])
sim_sample <- rbindlist(l, use.names = TRUE, fill=TRUE, idcol = NULL)
setorder(sim_sample, Year, Source, Dest)
sum(sim_sample$Parentage) #should be 37
##make a parentage matrix for each year
#mat2012 <- dcast(sim_sample[Year==2012][, Parentage := sum(Parentage), by=c("Year", "Source", "Dest")], Source ~ Dest, value.var="Parentage", fun.aggregate = sum)[Source %in% SurveyData[year==2012 & PropAnemSamp >0, site], -"Source"] 
#mat2012 <- as.matrix(rbind(mat2012, t(unique(unassigned[Year==2012][, num_sampled := sum(num_sampled), by=c("Year", "Dest")], by=c("Year", "Dest"))[, num_sampled]), use.names=F))
#
#mat2013 <- dcast(sim_sample[Year==2013][, Parentage := sum(Parentage), by=c("Year", "Source", "Dest")], Source ~ Dest, value.var="Parentage", fun.aggregate = sum)[Source %in% SurveyData[year==2013 & PropAnemSamp >0, site], -"Source"] 
#mat2013 <- as.matrix(rbind(mat2013, t(unique(unassigned[Year==2013][, num_sampled := sum(num_sampled), by=c("Year", "Dest")], by=c("Year", "Dest"))[, num_sampled]), use.names=F))
#
#mat2014 <- dcast(sim_sample[Year==2014][, Parentage := sum(Parentage), by=c("Year", "Source", "Dest")], Source ~ Dest, value.var="Parentage", fun.aggregate = sum)[Source %in% SurveyData[year==2014 & PropAnemSamp >0, site], -"Source"] 
#mat2014 <- as.matrix(rbind(mat2014, t(unique(unassigned[Year==2014][, num_sampled := sum(num_sampled), by=c("Year", "Dest")], by=c("Year", "Dest"))[, num_sampled]), use.names=F))
#
#mat2012_4 <- dcast(sim_sample[, Parentage := sum(Parentage), by=c("Year", "Source", "Dest")], Source ~ Dest, value.var="Parentage", fun.aggregate = sum)[Source %in% SurveyData[year %in% c(2012, 2013, 2014) & PropAnemSamp >0, site], -"Source"] 
#mat2012_4 <- as.matrix(rbind(mat2012_4, t(unique(unassigned[, total_sampled := sum(num_sampled), by="Dest"], by="Dest")[, total_sampled]), use.names=F))
#
#matNEM <- dcast(sim_sample[, Parentage := sum(Parentage), by=c("Monsoon", "Source", "Dest")][Monsoon=="NEM"], Source ~ Dest, value.var="Parentage", fun.aggregate = sum)[Source %in% SurveyData[year %in% c(2012, 2013, 2014) & PropAnemSamp >0, site], -"Source"] 
#matNEM <- as.matrix(rbind(matNEM, t(unique(unassigned[Monsoon=="NEM"][, total_sampled := sum(num_sampled), by="Dest"], by="Dest")[, total_sampled]), use.names=F))
#
#matSWM <- dcast(sim_sample[, Parentage := sum(Parentage), by=c("Monsoon", "Source", "Dest")][Monsoon=="SWM"], Source ~ Dest, value.var="Parentage", fun.aggregate = sum)[Source %in% SurveyData[year %in% c(2012, 2013, 2014) & PropAnemSamp >0, site], -"Source"] 
#matSWM <- as.matrix(rbind(matSWM, t(unique(unassigned[Monsoon=="SWM"][, total_sampled := sum(num_sampled), by="Dest"], by="Dest")[, total_sampled]), use.names=F))
#
#
#
##fit the kernels
#x <- list(Distances=Distances, Assignments=mat2012, Sampled_reefs=t(site_index[site %in% SurveyData[year==2012 & PropAnemSamp >0 , site], index]),
#                  Reef_sizes=Reef_sizes, Adult_sample_proportions=SurveyData[year==2012 & PropAnemSamp >0 , PropAnemSamp]) #put inputs into a list because that's the bbmle format
#Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))
#
#x <- list(Distances=Distances, Assignments=mat2013, Sampled_reefs=t(site_index[site %in% SurveyData[year==2013 & PropAnemSamp >0 , site], index]),
#                  Reef_sizes=Reef_sizes, Adult_sample_proportions=SurveyData[year==2013 & PropAnemSamp >0 , PropAnemSamp]) #put inputs into a list because that's the bbmle format
#Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))
#
#x <- list(Distances=Distances, Assignments=mat2014, Sampled_reefs=t(site_index[site %in% SurveyData[year==2014 & PropAnemSamp >0 , site], index]),
#                  Reef_sizes=Reef_sizes, Adult_sample_proportions=SurveyData[year==2014 & PropAnemSamp >0 , PropAnemSamp]) #put inputs into a list because that's the bbmle format
#Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))
#
#x <- list(Distances=Distances, Assignments=mat2012_4, Sampled_reefs=t(site_index[site %in% SurveyData[year==2014 & PropAnemSamp >0 , site], index]),
#                  Reef_sizes=Reef_sizes, Adult_sample_proportions=matrix(SurveyData[year==2014 & PropAnemSamp >0 , PropAnemSamp])) #put inputs into a list because that's the bbmle format
#Sim2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))
#
#x <- list(Distances=Distances, Assignments=matNEM, Sampled_reefs=t(site_index[site %in% SurveyData[year==2014 & PropAnemSamp >0 , site], index]),
#                  Reef_sizes=Reef_sizes, Adult_sample_proportions=matrix(SurveyData[year==2014 & PropAnemSamp >0 , PropAnemSamp])) #put inputs into a list because that's the bbmle format
#SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))
#
#x <- list(Distances=Distances, Assignments=matSWM, Sampled_reefs=t(site_index[site %in% SurveyData[year==2014 & PropAnemSamp >0 , site], index]),
#                  Reef_sizes=Reef_sizes, Adult_sample_proportions=matrix(SurveyData[year==2014 & PropAnemSamp >0 , PropAnemSamp])) #put inputs into a list because that's the bbmle format
#SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))
#
#
##store data from the fits in intermeadiate table, then append to master
#SimKernels_int <- data.table(TimeScale=character(), TimeID=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric(), iteration=numeric())[1:6]
#
#BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
#BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
#MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
#k_eval <- BestK2012
#theta_eval <- BestTheta2012
#Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
#Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
#
#SimKernels_int$TimeScale[1] <- "annual"
#SimKernels_int$TimeID[1] <- "2012"
#SimKernels_int$k[1] <-  BestK2012
#SimKernels_int$theta[1] <- BestTheta2012
#SimKernels_int$mdd[1] <- MDD2012
#SimKernels_int$med[1] <- Med2012
#SimKernels_int$dist90[1] <- Dist90_2012
#SimKernels_int$iteration[1] <- i
#
#BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
#BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
#MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
#k_eval <- BestK2013
#theta_eval <- BestTheta2013
#Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
#Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
#
#SimKernels_int$TimeScale[2] <- "annual"
#SimKernels_int$TimeID[2] <- "2013"
#SimKernels_int$k[2] <-  BestK2013
#SimKernels_int$theta[2] <- BestTheta2013
#SimKernels_int$mdd[2] <- MDD2013
#SimKernels_int$med[2] <- Med2013
#SimKernels_int$dist90[2] <- Dist90_2013
#SimKernels_int$iteration[2] <- i
#
#BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
#BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
#MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
#k_eval <- BestK2014
#theta_eval <- BestTheta2014
#Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
#Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
#
#SimKernels_int$TimeScale[3] <- "annual"
#SimKernels_int$TimeID[3] <- "2014"
#SimKernels_int$k[3] <-  BestK2014
#SimKernels_int$theta[3] <- BestTheta2014
#SimKernels_int$mdd[3] <- MDD2014
#SimKernels_int$med[3] <- Med2014
#SimKernels_int$dist90[3] <- Dist90_2014
#SimKernels_int$iteration[3] <- i
#
#BestK2012_4 <- as.numeric(coef(Sim2012_4Fit)[1])
#BestTheta2012_4 <- as.numeric(coef(Sim2012_4Fit)[2])
#MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
#k_eval <- BestK2012_4
#theta_eval <- BestTheta2012_4
#Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
#Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
#
#SimKernels_int$TimeScale[4] <- "interannual"
#SimKernels_int$TimeID[4] <- "2012-4"
#SimKernels_int$k[4] <-  BestK2012_4
#SimKernels_int$theta[4] <- BestTheta2012_4
#SimKernels_int$mdd[4] <- MDD2012_4
#SimKernels_int$med[4] <- Med2012_4
#SimKernels_int$dist90[4] <- Dist90_2012_4
#SimKernels_int$iteration[4] <- i
#
#BestKNEM <- as.numeric(coef(SimNEMFit)[1])
#BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
#MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
#k_eval <- BestKNEM
#theta_eval <- BestThetaNEM
#MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
#Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
#
#SimKernels_int$TimeScale[5] <- "season"
#SimKernels_int$TimeID[5] <- "NEM"
#SimKernels_int$k[5] <-  BestKNEM
#SimKernels_int$theta[5] <- BestThetaNEM
#SimKernels_int$mdd[5] <- MDDNEM
#SimKernels_int$med[5] <- MedNEM
#SimKernels_int$dist90[5] <- Dist90_NEM
#SimKernels_int$iteration[5] <- i
#
#BestKSWM <- as.numeric(coef(SimSWMFit)[1])
#BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
#MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
#k_eval <- BestKSWM
#theta_eval <- BestThetaSWM
#MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
#Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
#
#SimKernels_int$TimeScale[6] <- "season"
#SimKernels_int$TimeID[6] <- "SWM"
#SimKernels_int$k[6] <-  BestKSWM
#SimKernels_int$theta[6] <- BestThetaSWM
#SimKernels_int$mdd[6] <- MDDSWM
#SimKernels_int$med[6] <- MedSWM
#SimKernels_int$dist90[6] <- Dist90_SWM
#SimKernels_int$iteration[6] <- i
#
#l <- list(SimKernels, SimKernels_int)
#SimKernels <- rbindlist(l, use.names = TRUE, fill=TRUE, idcol = FALSE)
#setTxtProgressBar(pb, i)
#    
#    }
#close(pb)
#EndTime <- Sys.time()
#EndTime-StartTime
#
#
##write.csv(SimKernels, file="~/oceanography/script_output/KernelFits/Adjust_Survival_NoRecentRecruits/", row.names=F)



[1] TRUE

[1] 37

[1] TRUE

[1] 37

[1] 37

__Next step is to figure out what prop of the "other" and "CAI" source particles to sample, then sample them within the loop as well__

In [24]:
sum(mat2012)
sum(mat2013)
sum(mat2014)

[1] 65

[1] 178

[1] 222

In [83]:
test <- sim_sample[Year==2012][,  TotalPar :=sum(Parentage), by=c("Source", "Dest")][]#[, Parentage])
sum(test$TotalPar)

[1] 10

In [67]:
sim_sample[, c("Year", "Source", "Dest", "Monsoon", "Parentage")][order(Monsoon)]

Year,Source,Dest,Monsoon,Parentage
2013,Caridad Proper,Magbangon,NEM,1
2013,Poroc San Flower,Elementary School,NEM,1
2013,Sitio Baybayon,Sitio Baybayon,NEM,1
2013,Sitio Baybayon,Sitio Baybayon,NEM,1
2013,Elementary School,Magbangon,NEM,1
2013,Hicgop South,Hicgop South,NEM,1
2013,Tamakin Dacot,Tamakin Dacot,NEM,1
2013,Sitio Lonas,Elementary School,NEM,1
2013,Caridad Cemetery,Elementary School,NEM,1
2013,Sitio Lonas,Elementary School,NEM,1


In [21]:
sim_sample[Year==2013]#[, Parentage := sum(Parentage), by=c("Year", "Source", "Dest")][]

Year,Source,Dest,Monsoon,Parentage
2013,Cabatoan,Cabatoan,NEM,3
2013,Cabatoan,Cabatoan,SWM,3
2013,Caridad Cemetery,Caridad Cemetery,NEM,0
2013,Caridad Cemetery,Caridad Cemetery,SWM,0
2013,Caridad Cemetery,Magbangon,NEM,1
2013,Caridad Cemetery,Palanas,SWM,1
2013,Caridad Proper,Caridad Proper,NEM,0
2013,Caridad Proper,Caridad Proper,SWM,0
2013,Elementary School,Elementary School,NEM,2
2013,Elementary School,Elementary School,SWM,2
